In [ ]:
import scipy.special as sps 
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
X_INCREMENT = 0.001
L_FACTOR = 3

class PriceCurve():
    
    def __init__(self, L, l_factor = L_FACTOR, x_inc = X_INCREMENT):
        self.__L = L
        self.__x_inc = x_inc
        self.__l_factor = l_factor
        self.__x_val = self.__gen_x_val()
     
    def __gen_x_val(self, L = None, l_factor = None):      
        self.__l_factor = self.__l_factor if l_factor == None else l_factor
        self.__L = self.__L if L == None else L
        bound = self.get_bound()
        return np.arange(self.__x_inc, bound, self.__x_inc)
    
    def set_liquidity(self, L):  
        self.__L = L
        
    def set_factor(self, l_factor):  
        self.__l_factor = l_factor        
        
    def set_x_increment(self, x_inc):
        self.__x_inc = x_inc
        
    def get_bound(self):  
        return self.__l_factor*self.__L        
    
    def x_val(self):
        return self.__x_val
    
    def y_val(self, L = None, l_factor = None):
        self.__l_factor = self.__l_factor if l_factor == None else l_factor
        self.__L = self.__L if L == None else L
        self.__x_val = self.__gen_x_val(self.__L, self.__l_factor) 
        return self.__L**2/self.__x_val        

In [ ]:
class ConstantProductTrade():
    
    def __init__(self, L):
        self.__L = L
        
    def set_liquidity(self, L):
        self.__L = L
        
    def f(self, x):
        return self.__L**2/x  
    
    def price_swap_x(self, price, delta):
        a = price
        b = -price*delta
        c = -self.__L**2 
        return (-b + (b**2 - 4*a*c)**0.5)/(2*a) 

    def price_swap_y(self, price, x_swap):
        return price*x_swap 
    
    def swap_point(self, price, delta):
        x_swap = self.price_swap_x(price, delta)
        y_swap = self.price_swap_y(price, x_swap)
        return (x_swap,y_swap)   
    
    def random_delta(self, max_trade = 1):
        shape, scale = 1, max_trade/5
        trade = np.random.gamma(shape, scale, 1)
        return min(trade[0],max_trade)    

In [ ]:
class PlotPriceCurve():
    
    def __init__(self, L, cpt = None):
        self.__L = L
        self.__cpt = ConstantProductTrade(L) if cpt == None else cpt
        
    def set_liquidity(self, L):
        self.__L = L 
        
    def set_cpt(self, cpt): 
        self.__cpt = cpt
        
    def gen_data(self, l_factor = None):
        pCurve = PriceCurve(self.__L)
        y_val = pCurve.y_val()
        x_val = pCurve.x_val()
        bound = pCurve.get_bound()
        return x_val, y_val, bound
        
    def apply(self, new_bound = None):
        x_val, y_val, bound = self.gen_data()
        bound = bound if new_bound == None else new_bound
        print(bound)
        fig, ax = plt.subplots(figsize=(10, 10))
        plt.plot(x_val, y_val, label='curve')
        plt.ylim(0, bound)
        plt.xlim(0, bound)  
        
    def apply_next(self, L = None): 
        x_val, y_val, bound = self.gen_data()
        plt.plot(x_val, y_val, label='curve')
        
    def plot_trade(self, price, delta, cpt = None, p_lines = True):
        (x_swap,y_swap) = self.__cpt.swap_point(price, delta)  
        plt.scatter([x_swap], [y_swap], marker='o', color='green',s=25, label = 'swaps')
        if(p_lines): 
            plt.plot([self.__cpt.f(y_swap), x_swap],[y_swap, y_swap],'r--',lw=0.5)
            plt.plot([x_swap, x_swap],[y_swap, self.__cpt.f(x_swap)],'r--',lw=0.5)
        

In [ ]:
L = 15
p = 1

cpt = ConstantProductTrade(L)
pCurve = PlotPriceCurve(L, cpt)
pCurve.apply()

for k in range(25):
    delta = cpt.random_delta(20)
    pCurve.plot_trade(p, delta)
